In [1]:
import pandas as pd
import numpy as np
import statistics as st
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set(rc={"figure.figsize":(15,6)})
pd.pandas.set_option("display.max_columns",None)
%matplotlib inline

In [2]:
data = pd.read_csv("diabetes.csv")

In [3]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
data.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


Agenda:

How to select best Optimizer

How to select no. of nodes/neurons in a layers

How to select no. of layers

All in one model

In [5]:
#values  convert data in to numpy arrays
x = data.drop("Outcome",axis=1).values
y = data["Outcome"].values

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [7]:
x = scaler.fit_transform(x)

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=42)

In [9]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(614, 8)
(154, 8)
(614,)
(154,)


In [10]:
import tensorflow as tf

In [11]:
LAYERS = [
    tf.keras.layers.Dense(32,activation="relu",input_dim=8),
    tf.keras.layers.Dense(1,activation="sigmoid")
]
model = tf.keras.models.Sequential(LAYERS)

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [13]:
LOSS_function = "binary_crossentropy"
OPTIMIZER = "Adam"
METRICS = ["accuracy"]

In [14]:
model.compile(loss=LOSS_function,optimizer=OPTIMIZER,metrics=METRICS)

In [15]:
model.fit(X_train,y_train,epochs=10,batch_size=32,validation_data=(X_test,y_test))

Epoch 1/10
20/20 [==============================] - 1s 12ms/step - loss: 0.7239 - accuracy: 0.4267 - val_loss: 0.6890 - val_accuracy: 0.5325
Epoch 2/10
20/20 [==============================] - 0s 3ms/step - loss: 0.6675 - accuracy: 0.5912 - val_loss: 0.6485 - val_accuracy: 0.6558
Epoch 3/10
20/20 [==============================] - 0s 3ms/step - loss: 0.6278 - accuracy: 0.6726 - val_loss: 0.6169 - val_accuracy: 0.6429
Epoch 4/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5977 - accuracy: 0.6938 - val_loss: 0.5951 - val_accuracy: 0.6494
Epoch 5/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5740 - accuracy: 0.7101 - val_loss: 0.5792 - val_accuracy: 0.6753
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5541 - accuracy: 0.7231 - val_loss: 0.5696 - val_accuracy: 0.7078
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5391 - accuracy: 0.7329 - val_loss: 0.5591 - val_accuracy: 0.7338
Epoch 8/10
20/20 [=

In [16]:
#how To Select OPTIMIZER 
def build_model(hp):
    
    LAYERS = [
        tf.keras.layers.Dense(32,activation="relu",input_dim=8),
        tf.keras.layers.Dense(1,activation="sigmoid")
        ]
    model = tf.keras.models.Sequential(LAYERS)
    
    LOSS_function = "binary_crossentropy"
    OPTIMIZER = hp.Choice("optimizer",values = ["Adam","sgd","rmsprop","adadelta"])
    METRICS = ["accuracy"]
    model.compile(loss=LOSS_function,optimizer=OPTIMIZER,metrics=METRICS)
    
    return model
    

In [17]:
import keras_tuner as kt
tuner = kt.RandomSearch(build_model,
                       objective="val_accuracy",
                       max_trials= 5)


INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [18]:
tuner.search(X_train, y_train, epochs=5, validation_data = (X_test,y_test))

Trial 4 Complete [00h 00m 01s]
val_accuracy: 0.7142857313156128

Best val_accuracy So Far: 0.7142857313156128
Total elapsed time: 00h 00m 01s
INFO:tensorflow:Oracle triggered exit


In [23]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'Adam'}

In [26]:
model = tuner.get_best_models(num_models=1)[0]

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.fit(X_train, y_train, epochs=10,initial_epoch=6 ,validation_data = (X_test,y_test))

Epoch 7/10
20/20 [==============================] - 1s 12ms/step - loss: 0.5288 - accuracy: 0.7622 - val_loss: 0.5266 - val_accuracy: 0.7597
Epoch 8/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5111 - accuracy: 0.7655 - val_loss: 0.5128 - val_accuracy: 0.7857
Epoch 9/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4967 - accuracy: 0.7671 - val_loss: 0.5038 - val_accuracy: 0.7727
Epoch 10/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4865 - accuracy: 0.7671 - val_loss: 0.4986 - val_accuracy: 0.7792


In [29]:
## how TO select number of nodes ANN
def build_model(hp):
    units = hp.Int("units",min_value=8,max_value = 130,step=8)
    
    LAYERS = [
        tf.keras.layers.Dense(units=units,activation="relu",input_dim=8),
        tf.keras.layers.Dense(1,activation="sigmoid")
        ]
    model = tf.keras.models.Sequential(LAYERS)
    
    LOSS_function = "binary_crossentropy"
    OPTIMIZER = "Adam"
    METRICS = ["accuracy"]
    model.compile(loss=LOSS_function,optimizer=OPTIMIZER,metrics=METRICS)
    
    return model

In [30]:
tuner = kt.RandomSearch(build_model,
                       objective="val_accuracy",
                       max_trials= 5,
                       directory ="mydir" )

In [31]:
tuner.search(X_train, y_train, epochs=5, validation_data = (X_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 16s
INFO:tensorflow:Oracle triggered exit


In [33]:
tuner.get_best_hyperparameters()[0].values

{'units': 128}

In [34]:
# how to select numbar of layers
import tensorflow
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [36]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(128,activation="relu",input_dim=8))
    
    for i in range(hp.Int("num_layers",min_value = 1,max_value=10)):
        model.add(Dense(128,activation="relu"))
                  
    model.add(Dense(1,activation="sigmoid"))
    
    OSS_function = "binary_crossentropy"
    OPTIMIZER = "Adam"
    METRICS = ["accuracy"]
    model.compile(loss=LOSS_function,optimizer=OPTIMIZER,metrics=METRICS)
    
    return model

In [37]:
tuner = kt.RandomSearch(build_model,
                       objective="val_accuracy",
                       max_trials= 5,
                       directory ="mydir1" )

In [38]:
tuner.search(X_train, y_train, epochs=5, validation_data = (X_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 18s
INFO:tensorflow:Oracle triggered exit


In [47]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6}

In [48]:
model = tuner.get_best_models(num_models=1)[0]

In [49]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1152      
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 128)               16512     
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dense_5 (Dense)             (None, 128)               16512     
                                                                 
 dense_6 (Dense)             (None, 128)               1

In [56]:
# Combine all model and Final Function
def build_model(hp):
    
    model = Sequential()
    
    counter = 0
    
    for i in range(hp.Int("num_layers",min_value = 8,max_value = 10)):
        if counter == 0:
            model.add(Dense(hp.Int("unit"+str(i),min_value = 8,max_value = 130,step=8),
            activation=hp.Choice("activation"+str(i),values = ["relu","t"]),
            input_dim = 8))
            
        else:
            model.add(Dense(hp.Int("unit"+str(i),min_value = 8,max_value = 130,step=8),
            activation=hp.Choice("activation"+str(i),values = ["relu","t"])))
            
        counter += 1
        
    model.add(Dense(1,activation="sigmoid"))
        
    LOSS_function = "binary_crossentropy"
    OPTIMIZER = "Adam"
    METRICS = ["accuracy"]
    model.compile(loss=LOSS_function,optimizer=OPTIMIZER,metrics=METRICS)
    
    return model

In [61]:
tuner = kt.RandomSearch(build_model,
                       objective="val_accuracy",
                       max_trials= 5,
                       directory ="mydir2")

INFO:tensorflow:Reloading Tuner from mydir2\untitled_project\tuner0.json


In [63]:
tuner.search(X_train, y_train, epochs=1, validation_data = (X_test,y_test))


Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
8                 |8                 |num_layers
120               |64                |unit0
relu              |relu              |activation0
40                |80                |unit1
relu              |relu              |activation1
16                |72                |unit2
t                 |relu              |activation2
72                |88                |unit3
t                 |t                 |activation3
72                |24                |unit4
relu              |relu              |activation4
128               |128               |unit5
t                 |relu              |activation5
8                 |96                |unit6
relu              |relu              |activation6
40                |40                |unit7
t                 |t                 |activation7



Traceback (most recent call last):
  File "C:\Users\yash mohite\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_tuner\engine\base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\yash mohite\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_tuner\engine\base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\yash mohite\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_tuner\engine\tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "C:\Users\yash mohite\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_tuner\engine\tuner.py", line 213, in _build_and_fit_model
    model = self._try_build(hp)
  File "C:\Users\yash mohite\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_tuner\engine\tuner.py", line 155, 

RuntimeError: Number of consecutive failures excceeded the limit of 3.
Traceback (most recent call last):
  File "C:\Users\yash mohite\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_tuner\engine\base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\yash mohite\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_tuner\engine\base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\yash mohite\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_tuner\engine\tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "C:\Users\yash mohite\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_tuner\engine\tuner.py", line 213, in _build_and_fit_model
    model = self._try_build(hp)
  File "C:\Users\yash mohite\AppData\Local\Programs\Pyth

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [64]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1152      
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 128)               16512     
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dense_5 (Dense)             (None, 128)               16512     
                                                                 
 dense_6 (Dense)             (None, 128)               1

In [ ]:
model.fit(X_train, y_train, epochs=10,initial_epoch=6 ,validation_data = (X_test,y_test))